In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from utils.BBI_utils import mergeData, computeBBI, simpleLowpassFilter, getSuperelevation, getAdvisorySpeed, alignData
from utils.LRS_utils import NCATgetReferencePoints, getDistToMid, getReferenceCurve, NCATgetRadius
from utils.NCAT_processing import NCAT_processing
from utils.SR_processing import SR_processing
import math

print(gpd.__version__)

In [ ]:
def filter_crash(crash):
    crash.filter(items=['Road_Name', 'KABCO_Seve', 'Manner_of_', 'Location_a', 'Latitude', 'Longitude'])
    # Filters out all collision based crashes
    crash = crash[crash['Manner_of_'].eq('Not a Collision with Motor Vehicle')]
    # Filters out intersection crashes
    crash = crash[crash['Location_a'].str.contains('Non-Intersection') | crash['Location_a'].eq('Off Roadway')]
    return crash

In [ ]:
# Initialize all data in variables
loc = pd.read_csv(r'2021_03_11_07_36_21_506_loc.csv')
acc = pd.read_csv(r'2021_03_11_07_36_21_506_acc.csv')
phone_sr = SR_processing(inFiles=[r'2021_03_11_07_36_21_506_loc.csv',r'2021_03_11_07_36_21_506_acc.csv'])

combined_phone_data = phone_sr.gdf
road_17 = gpd.read_file('0017_D1_2/0017_D1_2.shp')

crash = pd.read_csv(r'Crashdata.csv')
# Chooses certain columns
crash = filter_crash(crash)
crash_gpd = gpd.GeoDataFrame(
    crash, geometry=gpd.points_from_xy(x=crash.Longitude, y=crash.Latitude), crs=4326
)
crash_gpd = crash_gpd.to_crs(road_17.crs)

In [ ]:
smartphone = gpd.GeoDataFrame(
    loc, geometry=gpd.points_from_xy(x=loc.longitude_dd, y=loc.latitude_dd)
)
road_17['speed'] = [0]
smartphone['speed'] = loc[' speed_ms']

smartphone = gpd.GeoDataFrame(smartphone, geometry=smartphone['geometry'])
smartphone = smartphone.set_crs('epsg:4326')
#smartphone = smartphone.to_crs({'init' :'epsg:4326'}) 


print(road_17.crs)
smartphone = smartphone.to_crs(road_17.crs)
road_17 = road_17.to_crs(smartphone.crs)
print(smartphone.crs)
smartphone.plot(column='speed');

combined = smartphone.append(road_17)
from shapely.geometry import Point
#print(road_17.geometry.project(Point(1,2)))
#print(road_17.geometry.interpolate(114994.071224))
#print(type(road_17.geometry))
sections = gpd.GeoSeries()
for x in np.arange(0, 10000, 50):
    point = road_17.geometry.interpolate(x)
    sections = sections.append(point)


num_sections = road_17.geometry.length/20 + 1

for index, row in combined_phone_data.iterrows():
    to_interpolate = road_17.geometry.project(Point(row['longitude_dd'], row['latitude_dd']))
    point = road_17.geometry.interpolate(to_interpolate)
#     print(point)
    sections = sections.append(point)
sections = gpd.GeoDataFrame(geometry=sections)
sections['speed'] = 5
combined = combined.append(sections)
# combined = combined.append(crash_points)
phone_sr.gdf['speed'] = 5
combined = combined.append(phone_sr.gdf)
combined.plot(column='speed')
# smartphone.plot()

In [ ]:
# Aggregates sensor data into uniform sections of the road 
# (i.e. take the mean of all rows where the points are in the interval 0-20 ft)
def standardize_sensors_to_road(sensor_data, road_linestring, interval=20, type='mean'):
    standardized = sensor_data.copy()
    standardized['section'] = standardized.apply(lambda row: round(road_linestring.project(row.geometry)/interval),axis=1)
    standardized = standardized.groupby('section').mean()
    # Name here refers to the column name, which is section
    standardized.geometry = standardized.apply(lambda row: road_linestring.interpolate(row.name * interval), axis=1)[0]
    return standardized

# Counts the number of crashes in each section and adds it in the 'crash_count' field
def add_crash_count(sectionized_phone_data, crash_gpd, max_crash_dist=100):
    # Left joins the nearest point in sectionized_phone_data ONLY if the distance between point + crash is less than max_crash_dist
    section_crashes = gpd.sjoin_nearest(sectionized_phone_data, crash_gpd, how='left', max_distance=max_crash_dist)
    # Aggregate # 
    print(section_crashes)
    sectionized_phone_data['crash_count'] = section_crashes.groupby(level=0).count()['Distance']
# Left join
sectionized_phone_data = standardize_sensors_to_road(combined_phone_data, road_17.geometry, interval=100)
add_crash_count(sectionized_phone_data, crash_gpd)
print(sectionized_phone_data)


# for index, row in s.iterrows():
#     if(row['dist'] == row['dist']):
#         print(row)
#         count += 1
# s = s.groupby(level=0).count()
for index, row in sectionized_phone_data.iterrows():
    print("bruh", row['crash_count'])
# print(count)
# print(s.geometry)
# print(sectionized_phone_data)

In [ ]:
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('camera/2021_06_03_14_57_18_660_cam_2021_06_03_14_57_18_660.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        # Display the resulting frame
        cv2.imshow('Frame',frame)

        # Press Q on keyboard to  exit
        key = cv2.waitKey(25)
        if key == 32:
            cv2.waitKey()
        elif key & 0xFF == ord('q'):
            break
        
  # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()


In [ ]:
from IPython.display import Video

Video("camera/2021_06_03_14_57_18_660_cam_2021_06_03_14_57_18_660.mp4")